In [1]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
import matplotlib.pyplot as plt
import PIL
from importlib import import_module
from pathlib import Path
from distutils import util

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_pil_image
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

### Contents
1. System settings
2. Dataset & Dataloader & Augmentation
3. Model
4. Optimizer & Loss func
5. Training & Inference

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        """
        Args:
            num_classes (int): 정답 label의 classification 갯수
        """
        super().__init__() # nn.module을 상속받음
        # input size : (b x 3 x 227 x 227)
        
        self.net = nn.Sequential( # nn.Sequential : model의 층을 연속적으로 쌓아주는 함수
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3에 나와 있는 hyperparameter 값 사용  
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False), # inplace=True : 계산한 값을 변수에 덮어씌움 / True 시 오류가 나서 False로 통일
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        self.init_bias()  # 아래 함수 참고

    def init_bias(self):
        for layer in self.net: # self.net에 정의된 layer에서
            if isinstance(layer, nn.Conv2d): # 해당 layer가 Conv2D layer 라면 
                nn.init.normal_(layer.weight, mean=0, std=0.01) # Normal distribution으로 weight initialization 진행
                nn.init.constant_(layer.bias, 0) # Bias를 모두 0으로 initialization

        nn.init.constant_(self.net[4].bias, 1) # 2번째 Conv2D layer bias를 1로 초기화
        nn.init.constant_(self.net[10].bias, 1) # 4번째 Conv2D layer의 bias를 1로 초기화
        nn.init.constant_(self.net[12].bias, 1) # 5번째 Conv2D layer의 bias를 1로 초기화

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 256 * 6 * 6)  # view는 tensor shape를 바꿔줌, FC layer로 넘겨주기 위해 output을 1차원으로 펴주는 역할
        return self.classifier(x)

In [3]:
def seed_everything(seed): # Reproducibility를 위한 seed 고정 작업
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)

# -- settings

use_cuda = torch.cuda.is_available() # if the system supports CUDA -> True
device = torch.device("cuda" if use_cuda else "cpu")

# -- dataset & augmentation

data_transform = transforms.Compose([
            transforms.Resize((227, 227)), # image tensor size를 227 x 227로 변환 (input에 넣어주기 위함)
            transforms.ToTensor(), # input을 tensor type으로 전환
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # image tensor 값들을 normalization
])

training_data = datasets.CIFAR10(
    root="data", # Download=True일 시, 데이터를 다운받을 경로 // Download=False일 시, 데이터가 존재하는 경로
    train=True,  # True -> Training set에서 data를 가져옴, False -> Test set에서 data를 가져옴
    download=False, # Data가 시스템 내부에 존재하는 지 여부
    transform=data_transform, # 위에서 정의한 data transformation을 적용
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.

test_data = datasets.CIFAR10(
    root="data",
    train=False, 
    download=False,
    transform=data_transform,
)


# 데이터로더를 생성합니다.

batch_size = 128 # 논문에 나와 있는 batch size

train_dataloader = DataLoader(dataset=training_data,
    batch_size=batch_size,
    pin_memory=use_cuda,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=True)
    
test_dataloader = DataLoader(dataset=test_data,
    batch_size=batch_size,
    pin_memory=use_cuda,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



Shape of X [N, C, H, W]: torch.Size([128, 3, 227, 227])
Shape of y: torch.Size([128]) torch.int64


In [4]:
# -- model

model = AlexNet().to(device)
model = torch.nn.DataParallel(model)

print(model)

DataParallel(
  (module): AlexNet(
    (net): Sequential(
      (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
      (1): ReLU()
      (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): ReLU()
      (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU()
      (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU()
      (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): 

In [5]:
optimizer = optim.SGD(
        params=model.parameters(),
        lr=0.01,
        momentum=0.9, # 논문에 나와 있는 SGD momentum 값 
        weight_decay=0.0005)

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [6]:
# Training

torch.cuda.empty_cache()

for epoch in range(100):
    model.train()

    loss_value = 0
    matches = 0

    for idx, train_batch in enumerate(train_dataloader):
        lr_scheduler.step()

        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = F.cross_entropy(outs, labels)

        loss.backward()
        optimizer.step()

        if idx % 100 == 0:
            loss, current = loss.item(), idx * len(X)
            size = len(train_dataloader.dataset)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        size = len(test_dataloader.dataset)
        num_batches = len(test_dataloader)

        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            test_pred = model(X)
            test_loss += F.cross_entropy(test_pred, y).item()
            correct += (torch.argmax(test_pred, dim=-1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


loss: 2.298815  [    0/50000]
loss: 2.305852  [12800/50000]
loss: 2.300820  [25600/50000]
loss: 2.299890  [38400/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302721 

loss: 2.307292  [    0/50000]
loss: 2.291421  [ 1600/50000]
loss: 2.309454  [ 3200/50000]
loss: 2.301529  [ 4800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302721 

loss: 2.299090  [    0/50000]
loss: 2.309455  [ 1600/50000]


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
inp = torch.tensor([1.0, 2.0, 3, 4, 5])

outplace_dropout = nn.Dropout(p=0.4)
print(inp)
output = outplace_dropout(inp)
print(output)
print(inp) # Notice that the input doesn't get changed here


inplace_droput = nn.Dropout(p=0.4, inplace=True)
print(inplace_droput(inp))
print(inp)

tensor([1., 2., 3., 4., 5.])
tensor([1.6667, 0.0000, 5.0000, 6.6667, 0.0000])
tensor([1., 2., 3., 4., 5.])
tensor([0.0000, 0.0000, 5.0000, 0.0000, 8.3333])
tensor([0.0000, 0.0000, 5.0000, 0.0000, 8.3333])


In [10]:
AlexNet().net[4]

Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))